In [2]:
import os
import sys
import json
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import FancyBboxPatch
import networkx as nx
import numpy as np
from enum import Enum
from typing import Dict, List, Tuple, Set
import pandas as pd
from datetime import datetime


# Project paths setup
project_root = os.path.abspath(".")  # Current directory
implementation_path = os.path.join(project_root, "Implementation")
app_path = os.path.join(project_root, "App")
new_semantics_path = os.path.join(project_root, "NewSemantics") 

# Add paths to sys.path
for path in [implementation_path, app_path, new_semantics_path]:
    if path not in sys.path:
        sys.path.append(path)

from Implementation.enums import ElementStatus, QualityStatus, LinkType, LinkStatus
from NewSemantics.goal_model import GoalModel
from NewSemantics.istar_processor import read_istar_model
from Ui.Layout import Layout

print("All libraries imported successfully!")


All libraries imported successfully!


In [3]:
# Global state initialization
if 'executed_events' not in globals():
    executed_events = []

if 'interface_created' not in globals():
    interface_created = False

if 'handler_debug' not in globals():
    handler_debug = False

if 'last_update' not in globals():
    last_update = datetime.utcnow()


In [4]:
def create_model_from_your_code():
    filepath = "Data/example_from_paper.txt"
    return read_istar_model(filepath)

def get_status_color_from_your_model(element_id):
    if element_id in model.qualities:
        status = model.qualities[element_id]
        if status == QualityStatus.UNKNOWN:
            return 'white'
        elif status == QualityStatus.FULFILLED:
            return 'lightgreen'
        elif status == QualityStatus.DENIED:
            return 'lightcoral'
    else:
        status = model.tasks.get(element_id) or model.goals.get(element_id)
        if status == ElementStatus.UNKNOWN:
            return 'white'
        elif status == ElementStatus.TRUE_FALSE:
            return 'lightgreen'
        elif status == ElementStatus.TRUE_TRUE:
            return 'lightblue'
    return 'white'

# Initialize model
model = create_model_from_your_code()
print("Model and helper functions initialized!")

Model and helper functions initialized!


In [ ]:
def create_complete_interface():
    """Create the complete interface with all functionality in one place"""
    global interface_created, handler_debug
    
    # Header
    header = widgets.HTML(f"""
    <div style='text-align: center; margin-bottom: 20px;'>
        <h1 style='color: #2E86AB; margin-bottom: 10px;'>A unified view - Interactive High-Level Business Requirements Evaluation</h1>
        <p style='font-size: 14px; color: #666;'>Using GoalModel class from: {implementation_path}</p>
        <p style='font-size: 12px; color: #888;'>Last Updated: {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')} UTC</p>
    </div>
    """)
    
    # Legend remains the same
    legend = widgets.HTML("""
    <div style='background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 15px; font-size: 12px; border: 1px solid #ccc;'>
    <div style='font-weight: bold; margin-bottom: 8px; text-align: center;'>Conventions</div>
    <div><strong>Colors:</strong> 🤍 Unknown | 🟢 Satisfied/Fulfilled | 🔵 Executed Pending | 🔴 Denied</div>
    <div><strong>Shapes:</strong> ☁️ Quality | ⭕ Goal | ⬡ Task | ⬜ Process Transition</div>
    </div>
    """)
    
    # Create output widgets
    trace_output = widgets.Output()
    status_output = widgets.Output()
    viz_output = widgets.Output()
    debug_output = widgets.Output() if handler_debug else None
    
    # Controls
    process_dropdown = widgets.Dropdown(
        options=list(model.event_mapping.keys()),
        value=list(model.event_mapping.keys())[0],
        description='Event:',
        disabled=False,
    )
    
    execute_button = widgets.Button(
        description='Execute Event',
        disabled=False,
        button_style='success',
        tooltip='Execute the selected event'
    )
    
    reset_button = widgets.Button(
        description='Reset Model',
        disabled=False,
        button_style='warning',
        tooltip='Reset the model to initial state'
    )
    
    controls = widgets.HBox([process_dropdown, execute_button, reset_button])

    # Update functions remain the same
    def update_trace():
        with trace_output:
            clear_output(wait=True)
            trace_html = """
            <div style='border: 2px solid #ccc; padding: 15px; margin: 10px; background-color: #f9f9f9;'>
                <h3>Trace Execution Timeline</h3>
            """
            if not executed_events:
                trace_html += "<p>No events executed yet. Select an event and click 'Execute Event' to start.</p>"
            else:
                trace_html += "<div style='display: flex; align-items: center; gap: 10px; font-size: 18px; font-weight: bold;'>"
                trace_html += "<span style='color: #666;'>trace ⟨</span>"
                for i, event in enumerate(executed_events):
                    if i > 0:
                        trace_html += "<span style='color: #666;'>,</span>"
                    trace_html += f"<span style='color: #2E86AB; margin: 0 5px;'>{event}</span>"
                trace_html += "<span style='color: #666;'>⟩</span></div>"
            trace_html += "</div>"
            display(HTML(trace_html))

    # The visualization update function remains the same
    def update_visualization():
        if handler_debug:
            print("Starting visualization update...")
        
        with viz_output:
            clear_output(wait=True)
            # Your existing visualization code remains unchanged
            # ... (keep all your existing visualization code here)
            
        if handler_debug:
            print("Visualization update completed")

    # Modified event handlers with duplicate prevention
    def execute_event_handler(b):
        global model, executed_events
        
        # Prevent duplicate executions
        if getattr(execute_button, '_processing', False):
            if handler_debug:
                print("Preventing duplicate execution")
            return
            
        execute_button._processing = True
        execute_button.disabled = True
        
        try:
            selected_event = process_dropdown.value
            
            if handler_debug:
                print(f"Processing event: {selected_event}")
            
            with status_output:
                clear_output(wait=True)
                print(f"Executing event: {selected_event}")
                executed_events.append(selected_event)
                model.process_event(selected_event)
                print(f"Event {selected_event} processed!")
                print("\nCurrent Status:")
                for quality_id, status in model.qualities.items():
                    print(f"  {quality_id}: {model._format_status(status)}")
            
            update_trace()
            update_visualization()
            
        finally:
            execute_button._processing = False
            execute_button.disabled = False
            if handler_debug:
                print("Event processing completed")

    def reset_model_handler(b):
        global model, executed_events
        
        # Prevent duplicate executions
        if getattr(reset_button, '_processing', False):
            if handler_debug:
                print("Preventing duplicate reset")
            return
            
        reset_button._processing = True
        reset_button.disabled = True
        
        try:
            if handler_debug:
                print("Starting model reset")
            
            model = create_model_from_your_code()
            executed_events = []
            
            with status_output:
                clear_output(wait=True)
                print("Model and trace reset to initial state")
            
            update_trace()
            update_visualization()
            
        finally:
            reset_button._processing = False
            reset_button.disabled = False
            if handler_debug:
                print("Reset completed")

    # Clear existing handlers and register new ones
    execute_button._click_handlers.callbacks.clear()
    reset_button._click_handlers.callbacks.clear()
    
    if handler_debug:
        print("Handlers cleared, registering new handlers...")
    
    execute_button.on_click(execute_event_handler)
    reset_button.on_click(reset_model_handler)
    
    if handler_debug:
        print("Handlers registered successfully")

    # Create interface layout
    controls_section = widgets.VBox([
        widgets.HTML("<h3>Controls</h3>"),
        controls,
        status_output
    ])
    
    main_content = widgets.VBox([
        controls_section,
        trace_output,
        widgets.HTML("<h3>Model Views</h3>"),
        viz_output
    ])
    
    if debug_output:
        main_content.children += (debug_output,)
    
    interface = widgets.VBox([header, legend, main_content])
    
    return interface

# Function to reset notebook state
def reset_notebook_state():
    """Reset all global state variables"""
    global interface_created, executed_events, model, handler_debug
    interface_created = False
    executed_events = []
    handler_debug = True  # Set to False in production
    if 'model' in globals():
        model = create_model_from_your_code()
    print("Notebook state reset completed")

# Main execution
if __name__ == "__main__":
    reset_notebook_state()
    
    if not interface_created:
        interface = create_complete_interface()
        display(interface)
        interface_created = True
        print("Interface created and displayed!")
    else:
        print("Interface already created. Run reset_notebook_state() to recreate.")

Notebook state reset completed
Handlers cleared, registering new handlers...
Handlers registered successfully


Interface created and displayed!
